<a href="https://colab.research.google.com/github/danielray54/DeepPulse/blob/main/DeepPulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepPulse
## An Uncertainty-aware Deep Neural Network for Heart Rate Estimations from Wrist-worn Photoplethysmography
> 2022 44th Annual International Conference of the IEEE Engineering in Medicine & Biology Society (EMBC)

- [Paper](https://ieeexplore.ieee.org/document/9871813) 

### Install Packages

In [ ]:
!pip install tensorflow-addons
!pip install uncertainty-toolbox

## Download Datasets

In [ ]:
#BAMI
!gdown 1g5gqh6vekEdi3ZT21Cdu_1fkfNjBeUOA

In [ ]:
#DALIA
!gdown 12DnrzMCV_otfU5_YUbedwRRcIyiHShIm

In [ ]:
#IEEE Test
!gdown 1PSciZgnXPlsYBMzR1Oj4TjcTk_LjL7TQ

#IEEE Train
!gdown 174KyqOiuhl3Prsrn29KgeMmIJVgYLSQK

### Imports

In [ ]:
#Import Necessary Packages
#util libs
import pickle
import os
import random
import time
#dl libs
import tensorflow as tf
import keras.backend as K
import tensorflow.keras as keras
from keras.callbacks import Callback
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow_addons as tfa
from tensorflow.keras import regularizers
#deeplearning addtional libs
import tensorflow_probability as tfp 
tfd = tfp.distributions
import uncertainty_toolbox as uct
#computation libs
import scipy.signal as sci_sig
import pandas as pd
from scipy import stats
import numpy as np
from sklearn.utils import shuffle
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
import fnmatch
import itertools
import re
from pathlib import Path
#display libs
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib as mpl
from matplotlib.transforms import Bbox
import seaborn as sns
from tqdm.notebook import tqdm_notebook, tqdm, trange
from google.colab import output

print("Tensorflow Version: ", tf.__version__)
print("Built with CUDA: ", tf.test.is_built_with_cuda())
print("CPUs: ", tf.config.list_physical_devices('CPU'))
print("GPUs: ", tf.config.list_physical_devices('GPU'))

### Experiment Variables

In [10]:
########### PREPROCESSING VARIABLES ##################
BAND_PASS_LOW = 0.5
BAND_PASS_HIGH = 4.5
BAND_PASS_ORDER = 2
RESAMPLE_FS = 64

########### ARCHITECTURAL VARIABLES ##################
SENSOR_FILTER_SIZE = 16
SENSOR_NUM_FILTER = 64
SENSOR_DROPOUT = 0.15 
SENSOR_CONV_TYPE = "Normal"
GLOBAL_FILTER_SIZE = 16
GLOBAL_NUM_FILTER = 128
GLOBAL_DROPOUT = 0.15
GLOBAL_CONV_TYPE = "Normal"
RECURRENT_UNITS = 32
RECURRENT_DROPOUT = 0.15
MAXPOOLING = 2

########### TRAINING VARIABLES ##################
NUM_EPOCHS = 200
BATCH_SIZE = 32
STRATIFIED = True
TEST_SIZE= 0.33
NUM_PREDICT_SAMPLES = 25
OPTIMISER = tf.optimizers.Nadam()
EXP_NAME = ["IEEE", "BAMI", "DALIA"]
PATH = '/content/drive/MyDrive/DeepPulse/Exp/'

########### OUTPUT VARIABLES ##################
DISPLAY = False
PUBLICATION = False

In [11]:
########### DATA ##################
infile = open("/content/BAMI_Data",'rb')
BAMI_Data = pickle.load(infile)
infile.close()
infile = open("/content/DaLia_Data",'rb')
DaLia_Data = pickle.load(infile)
infile.close()
infile = open("/content/IEEE_Test_Data",'rb')
IEEE_Test_Data = pickle.load(infile)
infile.close()
infile = open("/content/IEEE_Train_Data",'rb')
IEEE_Train_Data = pickle.load(infile)
infile.close()
IEEE_Data = IEEE_Train_Data + IEEE_Test_Data
DATA = [IEEE_Data, BAMI_Data, DaLia_Data]

In [12]:
def make_DeepPulse(SENSOR_FILTER_SIZE, SENSOR_NUM_FILTER, SENSOR_DROPOUT, 
                   GLOBAL_FILTER_SIZE, GLOBAL_NUM_FILTER, 
                   GLOBAL_DROPOUT, RECURRENT_UNITS, 
                   RECURRENT_DROPOUT, MAXPOOLING):
  keras.backend.clear_session()
  #=-=================== INPUTS ================================
  input_PPG = keras.Input(shape=(512, 1), name="PPG_Sigs")
  input_ACC = keras.Input(shape=(512, 3), name="ACC_Sigs")
  
  #=-=================== PPG ================================
  PPG_SENSOR_CONV_1 = Convolution(SENSOR_NUM_FILTER, SENSOR_FILTER_SIZE, 
                                  input_PPG, "PPG_SENSOR_1")
  PPG_SENSOR_CONV_2 = Convolution_MaxPool_DropOut(SENSOR_NUM_FILTER, 
                              SENSOR_FILTER_SIZE, MAXPOOLING, SENSOR_DROPOUT, 
                              PPG_SENSOR_CONV_1, "PPG_SENSOR_2")

  #=-=================== ACC ================================
  ACC_SENSOR_CONV_1 = Convolution(SENSOR_NUM_FILTER, SENSOR_FILTER_SIZE,
                                  input_ACC, "ACC_SENSOR_1")
  ACC_SENSOR_CONV_2 = Convolution_MaxPool_DropOut(SENSOR_NUM_FILTER, 
                              SENSOR_FILTER_SIZE, MAXPOOLING, SENSOR_DROPOUT, 
                              ACC_SENSOR_CONV_1, "ACC_SENSOR_2")
  
  #=-=================== MERGE ================================
  MERGE = layers.Concatenate(axis=2, name="MERGE")([PPG_SENSOR_CONV_2, 
                                                    ACC_SENSOR_CONV_2])

  #=-=================== GLOBAL ================================
  GLOBAL_CONV_1 = Convolution_MaxPool_DropOut(GLOBAL_NUM_FILTER, 
                              GLOBAL_FILTER_SIZE, MAXPOOLING, SENSOR_DROPOUT, 
                              MERGE, "GLOBAL_1")
  
  GLOBAL_CONV_2 = Convolution_MaxPool_DropOut(GLOBAL_NUM_FILTER, 
                                              GLOBAL_FILTER_SIZE, MAXPOOLING,
                                              GLOBAL_DROPOUT, GLOBAL_CONV_1, 
                                              "GLOBAL_2")
    
  #=-==========================================================
  RECURRENT_1 = LSTM_LayerNorm(RECURRENT_UNITS, RECURRENT_DROPOUT, True, 
                     GLOBAL_CONV_2, "RECURRENT_1")
  RECURRENT_2 = LSTM_LayerNorm(RECURRENT_UNITS, RECURRENT_DROPOUT, False, 
                     RECURRENT_1, "RECURRENT_2")
  #=-==========================================================
  dist = Prediction(GLOBAL_DROPOUT,"PREDICT",RECURRENT_2)

  DeepPulse = keras.Model(inputs = [input_PPG, input_ACC], outputs = dist, name="DeepPulse")
  return DeepPulse

# Helper Functions


In [13]:
def Convolution_MaxPool(SENSOR_NUM_FILTER, SENSOR_FEAT_FILTER_SIZE, MAX_POOLING, 
                       INPUT, NAME):
    """
    Keras DeepPulse Convolutional block with Maxpooling.

    Args:
        SENSOR_NUM_FILTER: The number of filters in the Convolutional block.
        SENSOR_FEAT_FILTER_SIZE: The filter size of the Convolutional block.
        MAX_POOLING: The max pooling for the Convolutional block.
        INPUT: The input to the Convolutional block.
        NAME: The name of the Convolutional block.

    Returns:
        x: The output of the Convolutional block.
    """
    x = layers.Conv1D(
          SENSOR_NUM_FILTER,
          SENSOR_FEAT_FILTER_SIZE,
          use_bias=False,
          name =  NAME +"_conv",
          padding="same")(INPUT)
    x = layers.ReLU(name =  NAME + "_relu")(x)
    x = layers.BatchNormalization(name =  NAME + "_batch_norm")(x)
    x = layers.MaxPool1D(MAX_POOLING, name =  NAME + "_max_pool")(x)
    return x

def Convolution_MaxPool_DropOut(SENSOR_NUM_FILTER, SENSOR_FEAT_FILTER_SIZE, 
                                MAX_POOLING, DROP_RATE, INPUT, NAME):
    """
    Keras DeepPulse Convolutional block with Maxpooling & Dropout.

    Args:
        SENSOR_NUM_FILTER: The number of filters in the Convolutional block.
        SENSOR_FEAT_FILTER_SIZE: The filter size of the Convolutional block.
        DROP_RATE: The dropout rate for the Convolutional block.
        MAX_POOLING: The max pooling for the Convolutional block.
        INPUT: The input to the Convolutional block.
        NAME: The name of the Convolutional block.

    Returns:
        x: The output of the Convolutional block.
    """
    x = layers.Conv1D(
          SENSOR_NUM_FILTER,
          SENSOR_FEAT_FILTER_SIZE,
          use_bias=False,
          name =  NAME +"_conv",
          padding="same")(INPUT)
    x = layers.ReLU(name =  NAME + "_relu")(x)
    x = layers.BatchNormalization(name =  NAME + "_batch_norm")(x)
    x = layers.AveragePooling1D(MAX_POOLING, name =  NAME + "_max_pool")(x)
    x = layers.Dropout(DROP_RATE, name =  NAME + "_dropout")(x, training=True)
    return x

def Convolution_DropOut(SENSOR_NUM_FILTER, SENSOR_FEAT_FILTER_SIZE, DROP_RATE, 
                        INPUT, NAME):
    """
    Keras DeepPulse Convolutional block with Dropout.

    Args:
        SENSOR_NUM_FILTER: The number of filters in the Convolutional block.
        SENSOR_FEAT_FILTER_SIZE: The filter size of the Convolutional block.
        DROP_RATE: The dropout rate for the Convolutional block.
        INPUT: The input to the Convolutional block.
        NAME: The name of the Convolutional block.

    Returns:
        x: The output of the Convolutional block.
    """
    x = layers.Conv1D(
          SENSOR_NUM_FILTER,
          SENSOR_FEAT_FILTER_SIZE,
          use_bias=False,
          name =  NAME +"_conv",
          padding="same")(INPUT)
    x = layers.ReLU(name =  NAME + "_relu")(x)
    x = layers.BatchNormalization(name =  NAME + "_batch_norm")(x)
    x = layers.Dropout(DROP_RATE, name =  NAME + "_dropout")(x, training=True)
    return x

def Convolution(SENSOR_NUM_FILTER, SENSOR_FEAT_FILTER_SIZE, INPUT, NAME):
    """
    Keras DeepPulse Convolutional block.

    Args:
        SENSOR_NUM_FILTER: The number of filters in the Convolutional block.
        SENSOR_FEAT_FILTER_SIZE: The filter size of the Convolutional block.
        DROP_RATE: The dropout rate for the Convolutional block.
        INPUT: The input to the Convolutional block.
        NAME: The name of the Convolutional block.

    Returns:
        x: The output of the Convolutional block.
    """
    x = layers.Conv1D(
          SENSOR_NUM_FILTER,
          SENSOR_FEAT_FILTER_SIZE,
          use_bias=False,
          name =  NAME +"_conv",
          padding="same")(INPUT)
    x = layers.ReLU(name =  NAME + "_relu")(x)
    x = layers.BatchNormalization(name =  NAME + "_batch_norm")(x)
    return x

def LSTM_LayerNorm(NUM_UNITS, lstm_do, return_seq, INPUT, NAME):
    """
    Keras DeepPulse Recurrent block.

    Args:
        NUM_UNITS: The number of LSTM units.
        lstm_do: The dropout rate for the LSTM block.
        return_seq: Boolean whether to return sequence or not.
        INPUT: The input to the Recurrent block.
        NAME: The name of the Recurrent block.

    Returns:
        dist: A Normal distribution.
    """
    x = layers.Bidirectional(layers.LSTM(NUM_UNITS, dropout=lstm_do,                          
        time_major=False,return_sequences=return_seq, 
        name =  NAME + "_lstm"), name=NAME + "_bidirectional")(INPUT, 
                                                               training=True)
    x = layers.LayerNormalization(name =  NAME + "_layer_norm")(x)
    return x

def Prediction(DROP_RATE, NAME, INPUT):
    """
    Keras DeepPulse Prediction module.

    Args:
        INPUT: The input to the prediction module.

    Returns:
        dist: A Normal distribution.
    """
    #x = layers.Dropout(DROP_RATE, name =  NAME + "_dropout")(INPUT, training=True)
    params = layers.Dense(2, name="Dense_1")(INPUT)
    dist = tfp.layers.DistributionLambda(normal_sp, name=
                                         "Distribution_1")(params)
    return dist

def butter_bandpass(lowcut, highcut, fs, order=5):
    """
    Creates a bandpass Butterworth IIR filter.
    Taken from:
    scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html

    Args:
        lowcut: The lower cutoff frequency.
        highcut: The higher cutoff frequency.
        fs: The sample rate.
        order: The order of the filter.

    Returns:
        b: Numerator polynomial of the IIR filter.
        a: Denominator polynomial of the IIR filter.
    """
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = sci_sig.butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    """
    Applies the bandpass Butterworth IIR filter to the data.
    Taken from:
    scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html

    Args:
        data: The signal to filter.
        lowcut: The lower cutoff frequency.
        highcut: The higher cutoff frequency.
        fs: The sample rate.
        order: The order of the filter.

    Returns:
        y: The output of the digital filter.
    """
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sci_sig.lfilter(b, a, data)
    return y

def resample_signal(signal_data, curr_fs, new_fs):
    """
    Resample the signal to a different sample rate.

    Args:
        signal_data: The signal to resample.
        curr_fs: The current sampling frequency.
        new_fs: The desired sampling frequency.

    Returns:
        resampled_signal: The output of the resampling
    """
    num_secs_in_signal = len(signal_data) / curr_fs
    num_samples_to_resample = round(num_secs_in_signal * new_fs)
    resampled_signal = sci_sig.resample(signal_data, num_samples_to_resample)
    return resampled_signal

def slidingWindow(sequence, winSize, step=1):
    """
    Creates a sliding window generator to iterate through input sequence.
    Taken from: https://stackoverflow.com/a/9878083

    Args:
        sequence: The signal to window.
        winSize: The size of sliding window.
        step: The size of overlap between windows.

    Returns:
        sequence[i:i+winSize]: sliding window generator
    """
    num_of_chunks = ((len(sequence) - winSize) / step) + 1
    for i in range(0, int(num_of_chunks) * step, step):
        yield sequence[i:i + winSize]

def preprocess(signal_data, fs, band_pass_low, band_pass_high, band_pass_order,
               resample_fs):
    """
    Preprocesses the signals: performs bandpass filtering then resampling.

    Args:
        signal_data: The raw signal.
        fs: The original sample rate of the signal.
        band_pass_low: The lower cutoff frequency of the bandpass filter.
        band_pass_high: The higher cutoff frequency of the bandpass filter.
        band_pass_order: The order of the bandpass filter.
        resample_fs: The desired sampling frequency.

    Returns:
        filtered_signal_resample: The preprocessed signal.
    """
    filtered_signal = butter_bandpass_filter(signal_data, band_pass_low, 
                                             band_pass_high, fs, 
                                             order=band_pass_order)
    filtered_signal_resample = resample_signal(filtered_signal, fs, resample_fs)
    return filtered_signal_resample

def flatten_list(t):
    """
    Flattens a 2D list to a 1D list.

    Args:
        t: The 2D List.

    Returns:
        a 1D list.
    """
    return [item for sublist in t for item in sublist]

def split_list(alist, wanted_parts=1):
    """
    Splits a list into n lists.
    taken from: https://stackoverflow.com/a/752562
    Args:
        alist: A list to be split.
        wanted_parts: The number of splits.

    Returns:
        list: A N-D list where n = wanted_parts
    """
    length = len(alist)
    return [alist[i * length // wanted_parts: (i + 1) * length // 
                  wanted_parts] for i in range(wanted_parts)]

def preprocess_dataset(dataset, band_pass_low, band_pass_high, band_pass_order, 
                       resample_fs):
    """
    Preprocesses the dataset of signals.

    Args:
        dataset: The dataset of raw signals.
        band_pass_low: The lower cutoff frequency of the bandpass filter.
        band_pass_high: The higher cutoff frequency of the bandpass filter.
        band_pass_order: The order of the bandpass filter.
        resample_fs: The desired sampling frequency.

    Returns:
        df: A Dataframe containing the preprocessed signals along with 
            additional information.
    """
    list_of_windows = []
    # get sample rates of signals
    fs_dataset_PPG = dataset[0]["PPG_fs"]
    fs_dataset_ACC = dataset[0]["ACC_fs"]
    # go through each session
    for idx, subject in tqdm_notebook(enumerate(range(len(dataset))), 
                                      desc='Sessions: ', total=len(dataset)):
        ppg1_signal_windowed = []
        accx_signal_windowed = []
        accy_signal_windowed = []
        accz_signal_windowed = []
        # ---------------PPG------------------------------------------
        # preprocess the PPG signal - filter, resample and window
        ppg_signal = preprocess(dataset[subject]["Raw_PPG_1"], fs_dataset_PPG, 
                                band_pass_low, band_pass_high, band_pass_order, 
                                resample_fs)
        for i in slidingWindow(ppg_signal, 8 * resample_fs, 2 * resample_fs):
            # for each window perform z-norm
            # applied on each window to prevent data leakage
            i = stats.zscore(i)
            ppg1_signal_windowed.append(i)
        # ---------------ACC------------------------------------------
        # preprocess the ACC signals - filter, resample and window
        accx_signal = preprocess(dataset[subject]["Raw ACC_X"], fs_dataset_ACC, 
                                 band_pass_low, band_pass_high, band_pass_order, 
                                 resample_fs)
        for i in slidingWindow(accx_signal, 8 * resample_fs, 2 * resample_fs):
            i = stats.zscore(i)
            accx_signal_windowed.append(i)
        accy_signal = preprocess(dataset[subject]["Raw ACC_Y"], fs_dataset_ACC,
                                 band_pass_low,band_pass_high, band_pass_order, 
                                 resample_fs)
        for i in slidingWindow(accy_signal, 8 * resample_fs, 2 * resample_fs):
            i = stats.zscore(i)
            accy_signal_windowed.append(i)
        accz_signal = preprocess(dataset[subject]["Raw ACC_Z"], fs_dataset_ACC, 
                                 band_pass_low,band_pass_high, band_pass_order,
                                 resample_fs)
        for i in slidingWindow(accz_signal, 8 * resample_fs, 2 * resample_fs):
            i = stats.zscore(i)
            accz_signal_windowed.append(i)
        # convert the lists to np.arrays
        ppg1_signal_windowed = np.array(ppg1_signal_windowed)
        accx_signal_windowed = np.array(accx_signal_windowed)
        accy_signal_windowed = np.array(accy_signal_windowed)
        accz_signal_windowed = np.array(accz_signal_windowed)
        # stack the ACC arrays into a tensor with shape
        # (Num of windows, length of windows, 3)
        acc_signals_stacked = np.stack((accx_signal_windowed,
                                        accy_signal_windowed, 
                                        accz_signal_windowed), axis=2)
        # Expand the dimension of the PPG tensor to have shape
        # (Num of windows, length of windows, 1)
        ppg1_signal_windowed = np.expand_dims(ppg1_signal_windowed, axis=2)
        # go thru each window and make a dict containing all the requried info.
        for window in range(len(ppg1_signal_windowed)):
            dict_entry = {
                "Signal_ID": idx,
                "Subject": dataset[subject]["Subject"],
                "Window_ID": window,
                "PPG": ppg1_signal_windowed[window],
                "ACC": acc_signals_stacked[window],
                "SNR": dataset[subject]["SNR"][window],
                "Activity": dataset[subject]["Protocol"][window],
                "Truth": dataset[subject]["truth_values"][window]
            }
            list_of_windows.append(dict_entry)
    # turn the list of dicts to a DF.
    df = pd.DataFrame(list_of_windows)
    return df

def get_test_results(model, df_test, fold, model_name, num_samples):
    """
    Get the results from the unseen session data.

    Args:
        model: The deep learning model.
        df_test: The dataframe of the unseen session data.
        fold: The fold of the LOSO CV.
        model_name: The name of the model.
        num_samples: The number of samples from the predictive distribution.

    Returns:
        df_calibration: A Dataframe containing the calibration metrics.
        df_uncertainty: A Dataframe containing the uncertainty metrics.

    """
    result_dict = []
    # create instance of model with the last layer of the model removed
    model_deeppulse = keras.Model(inputs=model.input, outputs=
                                  model.layers[-2].output)
    # for each window
    for index, row in tqdm_notebook(df_test.iterrows(), desc='Windows', 
                                    total=df_test.shape[0]):
        # repeat num_samples times to get samples from predictive distribution
        for d in range(num_samples):
            # get the model output
            x = model_deeppulse.predict([np.expand_dims(row["PPG"], axis=0), 
                                         np.expand_dims(row["ACC"], axis=0)],
                                        verbose=0)
            #make the model output a 1D list of floats
            output = [float(i) for i in x[0]]
            # convert model output to distribution
            distri = normal_sp_predict(output)
            # put info into dict to analyse
            dicts = {'Signal_ID': row["Signal_ID"],
                     'Subject': row["Subject"],
                     'Window': row["Window_ID"],
                     'SNR': row["SNR"],
                     'Activity': row["Activity"],
                     'Truth': row["Truth"],
                     'Model_ID': fold,
                     'Ensemble_ID': d,
                     'Mean_Value': distri.mean().numpy(),
                     'STD_Value': distri.stddev().numpy()}
            result_dict.append(dicts)
    # create DF from list of dicts
    df_calibration = pd.DataFrame(result_dict)
    # save df as csv
    file_name = model_name[:-3] + str("results_cali.csv")
    df_calibration.to_csv(file_name, index=False)
    # compute mean predicted value for each window
    groupby_mean_predict = df_calibration.groupby(["Signal_ID", "Subject", 
                                                   "Window", "Activity",
                                                   "SNR", "Truth"], 
                                            as_index=False)["Mean_Value"].mean()
    groupby_mean_predict["Absolute Error"] = abs(groupby_mean_predict["Truth"]
                                          - groupby_mean_predict["Mean_Value"])
    # compute Epistemic uncertainty
    groupby_epistemic = df_calibration.groupby(["Signal_ID", "Subject", 
                                                "Window"])['Mean_Value'].var() \
        .reset_index(name="Epistemic")
    epistemic_column = groupby_epistemic["Epistemic"]
    df_uncertainty = pd.concat([groupby_mean_predict, epistemic_column], axis=1)
    # compute Aleotoric uncertainty
    groupby_aleotoric = df_calibration.groupby(["Signal_ID", "Subject", 
                                              "Window"])['STD_Value'].median() \
        .reset_index(name="Aleotoric")
    aleotoric_column = groupby_aleotoric["Aleotoric"]
    df_uncertainty = pd.concat([df_uncertainty, aleotoric_column], axis=1)
    # compute predictive uncertainty
    df_uncertainty["Predictive"] = df_uncertainty["Epistemic"] + df_uncertainty["Aleotoric"]
    df_uncertainty.rename(columns={'Mean_Value': 'Predictive_Mean'}, inplace=True)
    # save df as csv
    file_name = model_name[:-3] + str("results_uncert.csv")
    df_uncertainty.to_csv(file_name, index=False)
    return df_calibration, df_uncertainty

def get_percentage_overlap_of_fold_sets(list_of_windows):
    """
    Get the percentage overlap of the set of data used for each fold, i.e. the training set/ the val set.
    "percentage" take from: https://stackoverflow.com/a/29929179

    Args:
        list_of_windows: A list of lists containing all the windows in the set of data used for each fold.

    Returns:
        data: A Dataframe containing the percentage overlap for each fold.
        matrix: The upper triangle of the Dataframe.

    """
    dict_of_percentages = []
    # get all possible combinations of fold into a list of tuples
    x = list(itertools.product(range(len(list_of_windows)), repeat=2))
    # for each tuple
    for i in range(len(x)):
        # split the lists into a list for each fold in the tuple
        first_list = x[i][0]
        second_list = x[i][1]
        # compute percentage overlap between the two lists
        percentage = len(set(list_of_windows[first_list]) & set(list_of_windows[second_list])) / \
                     float(len(set(list_of_windows[first_list]) | set(list_of_windows[second_list]))) * 100
        # store result in dict
        result = {
            "Fold X": first_list,
            "Fold Y": second_list,
            "percentage": percentage
        }
        dict_of_percentages.append(result)
    # store in df
    df = pd.DataFrame(dict_of_percentages)
    data = df.pivot("Fold X", "Fold Y", "percentage")
    # The upper triangle of the data
    matrix = np.triu(data)
    return data, matrix

def get_dataset_splits_loso(DATASET, stratified, test_sze, test_subject):
    """
    Get the train, validation and test sets for the current fold

    Args:
        DATASET: The dataframe of all windows.
        stratified: Whether the train/val sets are stratified on activity type.
        test_sze: The percentage of the data used for validation set.
        test_subject: The session ID for the test set.

    Returns:
        df_train: A Dataframe containing the training data.
        df_val: A Dataframe containing the validation data.
        df_test: A Dataframe containing the test data.

    """
    # get the windows that are associated with the test subject
    df_test = shuffle(DATASET.loc[DATASET['Signal_ID'] == test_subject])
    # get the train + val sets either stratified or not
    if stratified:
        df_train, df_val = train_test_split(DATASET, test_size=test_sze, shuffle=True, stratify=DATASET['Activity'])
    else:
        df_train, df_val = train_test_split(DATASET, test_size=test_sze, shuffle=True)
    return df_train, df_val, df_test


def display_training_performance(df_result, history, eval_history, fold, path, display, publication):
    """
    Display the training performance of the model on the fold of data.

    Args:
        df_result: The dataframe of all windows with labels of the set name (i.e. train set).
        history: The training history dict.
        eval_history: The testing history dict.
        fold: The current fold of the LOSO CV.
        path: The path of the experiment.
        display: Whether to display inline during training.
        publication: Whether to export as svg or png format.
    """
  
    if publication:
        fig_format = 'svg'
    else:
        fig_format = 'png'
    fig = plt.figure(figsize=(40, 25))
    my_suptitle = fig.suptitle('Training for Fold ' + str(fold), y=1.05)
    gs = GridSpec(nrows=3, ncols=5)
    # Bar chart of the size differences of the data splits
    ax0 = fig.add_subplot(gs[0, 0])
    ax0.title.set_text("Size of Dataset Splits")
    sns.countplot(ax=ax0, x="Dataset", data=df_result)
    ax0.set_xticklabels(ax0.get_xticklabels(), rotation=0)

    # Graph showing the distribution of the truth values for each split of data
    ax1 = fig.add_subplot(gs[0, 1])
    ax1.title.set_text("Distribution of Truth Values")
    sns.kdeplot(x='Truth', fill=False, data=df_result, hue="Dataset", ax=ax1)

    # Graph showing the distribution of the Signal to Noise Ratios for each split of data
    ax2 = fig.add_subplot(gs[0, 2])
    ax2.title.set_text("Distribution of Estimated Signal to Noise Ratio")
    sns.kdeplot(x='SNR', fill=False, data=df_result, hue="Dataset", ax=ax2)

    # Graph showing the distribution of the Activity Types for each split of data
    ax3 = fig.add_subplot(gs[0, 3])
    ax3.title.set_text("Distribution of Activity Types")
    sns.countplot(x="Activity", hue="Dataset", data=df_result, ax=ax3)
    ax3.set_xticklabels(ax3.get_xticklabels(), rotation=75)

    # Graph showing the Learning Rate Whilst Training
    ax4 = fig.add_subplot(gs[0, 4])
    ax4.title.set_text("Learning Rate Whilst Training")
    ax4.plot(history.history['lr'])
    ax4.set(xlabel="Epoch", ylabel="Learning Rate")

    # Graph showing the MAE Learning Curve
    ax5 = fig.add_subplot(gs[1, 0:4])
    ax5.title.set_text("Learning Curve (MAE)")
    ax5.plot(history.history['mae'], label="Train")
    ax5.plot(history.history['val_mae'], label="Validation")
    ax5.plot([eval_history[1]] * len(history.history['mae']), label="Test: " + str(round(eval_history[1], 3)))
    ax5.axvline(x=np.argmin(history.history['val_loss']), color='k', linestyle='--',
                label="Saved Model @ Epoch: " + str(np.argmin(history.history['val_loss'])))
    ax5.set(xlabel="Epoch", ylabel="MAE")
    ax5.legend(loc='upper right')

    # Graph showing the NLL Learning Curve
    ax6 = fig.add_subplot(gs[2, 0:4])
    ax6.title.set_text("Learning Curve (NLL)")
    ax6.plot(history.history['loss'], label="Train")
    ax6.plot(history.history['val_loss'], label="Validation")
    ax6.plot([eval_history[0]] * len(history.history['loss']), label="Test: " + str(round(eval_history[0], 3)))
    ax6.axvline(x=np.argmin(history.history['val_loss']), color='k', linestyle='--',
                label="Saved Model @ Epoch: " + str(np.argmin(history.history['val_loss'])))
    ax6.set_yscale('log')
    ax6.set(xlabel="Epoch", ylabel="NLL")
    ax6.legend(loc='upper right')

    # Bar chart showing the performance of MAE @ model checkpoint
    ax7 = fig.add_subplot(gs[1, 4:5])
    ax7.title.set_text("Performance of Learning (MAE)")
    name = ["Train", "Validate", "Test"]
    lii = [history.history['mae'][np.argmin(history.history['val_loss'])],
           history.history['val_mae'][np.argmin(history.history['val_loss'])], round(eval_history[1], 3)]
    sns.barplot(ax=ax7, x=name, y=lii)
    ax7.set(xlabel="Performance @ " + str(np.argmin(history.history['val_loss'])), ylabel="MAE")
    
    # Bar chart showing the performance of NLL @ model checkpoint
    ax8 = fig.add_subplot(gs[2, 4:5])
    ax8.title.set_text("Performance of Learning (NLL)")
    name = ["Train", "Validate", "Test"]
    lii = [history.history['loss'][np.argmin(history.history['val_loss'])],
           history.history['val_loss'][np.argmin(history.history['val_loss'])], round(eval_history[0], 3)]
    sns.barplot(ax=ax8, x=name, y=lii)
    ax8.set(xlabel="Performance @ Epoch" + str(np.argmin(history.history['val_loss'])), ylabel="NLL")
    
    gs.tight_layout(fig)
    # either save file as svg or png
    fig_name = path + '/fold' + str(fold) + 'training.' + fig_format
    fig.savefig(fig_name, format=fig_format, dpi=fig.dpi,
                bbox_inches='tight', bbox_extra_artists=[my_suptitle])
    # display graph during execution or not
    if display:
        plt.show()
        time.sleep(10)
        plt.close(fig)
        output.clear()

def display_training_performance_concrete(df_result, history, eval_history, fold, path, model, display, publication):
    """
    Display the training performance of the model on the fold of data using concrete dropout.

    Args:
        df_result: The dataframe of all windows with labels of the set name (i.e. train set).
        history: The training history dict.
        eval_history: The testing history dict.
        fold: The current fold of the LOSO CV.
        path: The path of the experiment.
        model: The trained model for that fold.
        display: Whether to display inline during training.
        publication: Whether to export as svg or png format.
    """
    fig = plt.figure(figsize=(40, 25))
    my_suptitle = fig.suptitle('Training for Fold ' + str(fold), y=1.05)
    gs = GridSpec(nrows=4, ncols=5)
    # Bar chart of the size differences of the data splits
    ax0 = fig.add_subplot(gs[0, 0])
    ax0.title.set_text("Size of Dataset Splits")
    sns.countplot(ax=ax0, x="Dataset", data=df_result)
    ax0.set_xticklabels(ax0.get_xticklabels(), rotation=0)
    # Graph showing the distribution of the truth values for each split of data
    ax1 = fig.add_subplot(gs[0, 1])
    ax1.title.set_text("Distribution of Truth Values")
    sns.kdeplot(x='Truth', fill=False, data=df_result, hue="Dataset", ax=ax1)
    #Graph showing the distribution of the Signal to Noise Ratios for each split of data
    ax2 = fig.add_subplot(gs[0, 2])
    ax2.title.set_text("Distribution of Estimated Signal to Noise Ratio")
    sns.kdeplot(x='SNR', fill=False, data=df_result, hue="Dataset", ax=ax2)
    # Graph showing the distribution of the Activity Types for each split of data
    ax3 = fig.add_subplot(gs[0, 3])
    ax3.title.set_text("Distribution of Activity Types")
    sns.countplot(x="Activity", hue="Dataset", data=df_result, ax=ax3)
    ax3.set_xticklabels(ax3.get_xticklabels(), rotation=75)
    #Graph showing the Learning Rate Whilst Training
    ax4 = fig.add_subplot(gs[0, 4])
    ax4.title.set_text("Learning Rate Whilst Training")
    ax4.plot(history.history['lr'])
    ax4.set(xlabel="Epoch", ylabel="Learning Rate")
    #Graph showing the MAE Learning Curve
    ax5 = fig.add_subplot(gs[1, 0:4])
    ax5.title.set_text("Learning Curve (MAE)")
    ax5.plot(history.history['mae'], label="Train")
    ax5.plot(history.history['val_mae'], label="Validation")
    ax5.plot([eval_history[1]] * len(history.history['mae']), label="Test: " + str(round(eval_history[1], 3)))
    ax5.axvline(x=np.argmin(history.history['val_loss']), color='k', linestyle='--',
                label="Saved Model @ Epoch: " + str(np.argmin(history.history['val_loss'])))
    ax5.set(xlabel="Epoch", ylabel="MAE")
    ax5.legend(loc='upper right')
    #Graph showing the NLL Learning Curve
    ax6 = fig.add_subplot(gs[2, 0:4])
    ax6.title.set_text("Learning Curve (NLL)")
    ax6.plot(history.history['loss'], label="Train")
    ax6.plot(history.history['val_loss'], label="Validation")
    ax6.plot([eval_history[0]] * len(history.history['loss']), label="Test: " + str(round(eval_history[0], 3)))
    ax6.axvline(x=np.argmin(history.history['val_loss']), color='k', linestyle='--',
                label="Saved Model @ Epoch: " + str(np.argmin(history.history['val_loss'])))
    ax6.set_yscale('log')
    ax6.set(xlabel="Epoch", ylabel="NLL")
    ax6.legend(loc='upper right')
    # Bar chart showing the performance of MAE @ model checkpoint
    ax7 = fig.add_subplot(gs[1, 4:5])
    ax7.title.set_text("Performance of Learning (MAE)")
    name = ["Train", "Validate", "Test"]
    lii = [history.history['mae'][np.argmin(history.history['val_loss'])],
           history.history['val_mae'][np.argmin(history.history['val_loss'])], round(eval_history[1], 3)]
    sns.barplot(ax=ax7, x=name, y=lii)
    ax7.set(xlabel="Performance @ " + str(np.argmin(history.history['val_loss'])), ylabel="MAE")
    # Bar chart showing the performance of NLL @ model checkpoint
    ax8 = fig.add_subplot(gs[2, 4:5])
    ax8.title.set_text("Performance of Learning (NLL)")
    name = ["Train", "Validate", "Test"]
    lii = [history.history['loss'][np.argmin(history.history['val_loss'])],
           history.history['val_loss'][np.argmin(history.history['val_loss'])], round(eval_history[0], 3)]
    sns.barplot(ax=ax8, x=name, y=lii)
    ax8.set(xlabel="Performance @ Epoch" + str(np.argmin(history.history['val_loss'])), ylabel="NLL")
    #Graph showing the Dropout Learning Curve
    ax9 = fig.add_subplot(gs[3, 0:4])
    ax9.title.set_text("Learning Curve (Dropout)")
    drop_name = np.array([layer.name for layer in model.layers if hasattr(layer, 'p_logit')])
    for idx, do in enumerate(drop_name):
        ax9.plot(history.history[do], label=str(do))
    ax9.axvline(x=np.argmin(history.history['val_loss']), color='k', linestyle='--',
                label="Saved Model @ Epoch: " + str(np.argmin(history.history['val_loss'])))
    ax9.set(xlabel="Epoch", ylabel="Drop Rate")
    ax9.legend(loc='upper right')
    # Bar chart showing the learnt dropout rates @ model checkpoint
    ax10 = fig.add_subplot(gs[3, 4:5])
    ax10.title.set_text("Dropout Rate Learnt for Each Layer")
    p_logits = np.array([K.eval(layer.p_logit) for layer in model.layers if hasattr(layer, 'p_logit')])
    sns.barplot(
        x=drop_name,
        y=flatten_list(tf.nn.sigmoid(p_logits).numpy()),
        ax=ax10)
    ax10.set(xlabel="Layer Name", ylabel="Drop Rate")
    ax10.set_xticklabels(ax10.get_xticklabels(), rotation=45)

    gs.tight_layout(fig)
    # either save file as svg or png
    if publication:
        fig_format = 'svg'
    else:
        fig_format = 'png'
    fig_name = path + '/fold' + str(fold) + 'training.' + fig_format
    fig.savefig(fig_name, format=fig_format, dpi=fig.dpi,
                bbox_inches='tight', bbox_extra_artists=[my_suptitle])
    # display graph during execution or not
    if display:
        plt.show()
        time.sleep(10)
        plt.close(fig)
        output.clear()

def display_calibration_performance(calibration_df, uncert_df, fold, path, display, publication):
    """
    Display the uncertainty & uncertainty calibration performance of the model on the fold of data.

    Args:
        calibration_df: A Dataframe containing the calibration metrics.
        uncert_df: A Dataframe containing the uncertainty metrics.
        fold: The current fold of the LOSO CV.
        path: The path of the experiment.
        display: Whether to display inline during training.
        publication: Whether to export as svg or png format.
    """
    # convert df columns to np arrays
    y_true = calibration_df['Truth'].to_numpy()
    # x = calibration_df['Window'].to_numpy()
    y_pred = calibration_df['Mean_Value'].to_numpy()
    y_std = calibration_df['STD_Value'].to_numpy()
    # Set random seed
    np.random.seed(11)
    fig = plt.figure(figsize=(40, 40))
    my_suptitle = fig.suptitle('Calibration for Fold ' + str(fold), y=1.05)
    gs = GridSpec(nrows=3, ncols=3)
    # graph showing the predicted values against the truth along with activity type, SNR and both uncertainty metrics
    ax010 = fig.add_subplot(gs[0, 0:3])
    # get the start indexes of each activity for the session
    start_indexes, activity = repeating_values(uncert_df["Activity"])
    colours = ['#42d4f4', '#f032e6', '#fabed4', '#469990', '#dcbeff', '#9A6324', '#fffac8',
               '#800000', '#aaffc3', '#000075', '#a9a9a9']
    # get upper and lower bands of the uncertainty terms for each window
    activities = uncert_df["Activity"].unique()
    pred = list(uncert_df["Predictive_Mean"])
    aleo = list(uncert_df["Aleotoric"])
    epi = list(uncert_df["Epistemic"])
    lower_aleo = np.array(pred) - np.array(aleo)
    upper_aleo = np.array(pred) + np.array(aleo)
    lower_epi = np.array(pred) - np.array(epi)
    upper_epi = np.array(pred) + np.array(epi)

    ax010.plot(range(len(uncert_df["Truth"])), uncert_df["Truth"])
    x_ax = np.arange(0, len(uncert_df["Predictive_Mean"]), 1)
    # plot Predicted values
    ax010.plot(x_ax, uncert_df["Predictive_Mean"], '-', label="Predicted", color='#e6194B')
    # plot truth values
    ax010.plot(x_ax, uncert_df["Truth"], '-', label="Truth", color='#000000')
    ax020 = ax010.twinx()
    # plot SNR values
    ax020.plot(x_ax, uncert_df["SNR"], '-', label="Est. SNR", color='#3cb44b')
    # plot activity type
    for idx, act in enumerate(activity):
        if idx + 1 == len(activity):
            ax010.axvspan(start_indexes[idx], len(uncert_df["Truth"]),
                          color=colours[int(np.where(act == activities)[0])],
                          alpha=0.1, label=activities[int(np.where(act == activities)[0])])
        else:
            ax010.axvspan(start_indexes[idx], start_indexes[idx + 1],
                          color=colours[int(np.where(act == activities)[0])],
                          alpha=0.1, label=activities[int(np.where(act == activities)[0])])
    # plot Aleotoric Uncertainty
    ax010.fill_between(x_ax, lower_aleo, upper_aleo,
                       color='#4363d8', alpha=0.5, label="Aleotoric Uncertainty")
    # plot Epistemic Uncertainty
    ax010.fill_between(x_ax, lower_epi, upper_epi,
                       color='#f58231', alpha=0.5, label="Epistemic Uncertainty")

    lines, labels = ax010.get_legend_handles_labels()
    lines2, labels2 = ax020.get_legend_handles_labels()
    by_label = dict(zip(labels, lines))
    by_label.update(zip(labels2, lines2))
    ax010.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                 ncol=4, mode="expand", borderaxespad=0.)
    ax010.set_xlabel('Windows')
    ax010.set_ylabel('Beats Per Minute')
    ax020.set_ylabel('Signal to Noise Ratio (dB)')
    # Plot calibration of predictive Uncertainty
    ax3 = fig.add_subplot(gs[1, 0])
    uct.plot_calibration(y_pred, y_std, y_true, ax=ax3)
    # Plot adversarial group calibration of predictive Uncertainty
    ax4 = fig.add_subplot(gs[1, 1])
    uct.plot_adversarial_group_calibration(y_pred, y_std, y_true, ax=ax4)
    # Plot sharpness of predictive Uncertainty
    ax5 = fig.add_subplot(gs[1, 2])
    uct.plot_sharpness(y_std, ax=ax5)
    # Plot boxplots of Epistemic Uncertainty for each activity type
    ax6 = fig.add_subplot(gs[2, 0])
    sns.boxplot(x='Activity', y='Epistemic', data=uncert_df, showfliers=False, orient='v', ax=ax6)
    ax65 = ax6.twinx()
    sns.countplot(x='Activity', data=uncert_df, alpha=0.35, ax=ax65)
    # Plot boxplots of Aleotoric Uncertainty for each activity type
    ax7 = fig.add_subplot(gs[2, 1])
    sns.boxplot(x='Activity', y='Aleotoric', data=uncert_df, showfliers=False, orient='v', ax=ax7)
    # plot relationship between Epistemic & Aleotoric Uncertainty
    ax8 = fig.add_subplot(gs[2, 2])
    sns.scatterplot(x='Aleotoric', y='Epistemic', data=uncert_df, ax=ax8)
    gs.tight_layout(fig)
    # either save file as svg or png
    if publication:
        fig_format = 'svg'
    else:
        fig_format = 'png'
 
    fig_plot_name = 'calibration.'    
    fig_name = path + '/fold' + str(fold) + fig_plot_name + fig_format
    fig.savefig(fig_name, format=fig_format, dpi=fig.dpi, bbox_inches='tight',
                bbox_extra_artists=[my_suptitle])
    # display graph during execution or not
    if display:
        plt.show()
        time.sleep(10)
        plt.close(fig)
        output.clear()

def display_overall_performance(path, unique_check_train, unique_check_val, 
                                unique_check_test, display, publication):
    """
    Display the overall performance of the model on all the fold of data.
    unique_check_train, unique_check_val, unique_check_test,
    Args:
        path: The path of the experiment.
        unique_check_train: list of list containing all windows used for training for each fold.
        unique_check_val: list of list containing all windows used for validation for each fold.
        unique_check_test: list of list containing all windows used for testing for each fold.
        display: Whether to display inline during training.
        publication: Whether to export as svg or png format.
    """
    all_uncert_df, all_calibration_df, df_session_specific, df_overall = get_overall_stats(path)
    # get calibration vals to numpy arrays
    y_true = all_calibration_df['Truth'].to_numpy()
    y_pred = all_calibration_df['Mean_Value'].to_numpy()
    y_std = all_calibration_df['STD_Value'].to_numpy()
    # Set random seed
    np.random.seed(11)
    fig = plt.figure(figsize=(50, 50))

    my_suptitle = fig.suptitle('Overall Results', y=1.05)
    gs = GridSpec(nrows=6, ncols=4)
    # Plot MAE for each session 
    ax001 = fig.add_subplot(gs[0, 0])
    all_uncert_df.groupby("Signal_ID")['Absolute Error'].mean().plot.bar(ax=ax001)
    ax001.axhline(np.mean(all_uncert_df.groupby("Signal_ID")['Absolute Error'].mean()), ls='--',
                  label="Average @" + str(
                      round(np.mean(all_uncert_df.groupby("Signal_ID")['Absolute Error'].mean()),2)) + ' BPM')
    ax001.set_ylabel('Mean Absolute Error')
    ax001.legend(loc='best')
    ax001.set_title("Mean Absolute Error By Session")
    # Plot MAX AE for each session
    ax002 = fig.add_subplot(gs[0, 1])
    all_uncert_df.groupby("Signal_ID")['Absolute Error'].max().plot.bar(ax=ax002)
    ax002.axhline(np.mean(all_uncert_df.groupby("Signal_ID")['Absolute Error'].max()), ls='--',
                  label="Average @" + str(
                      round(np.mean(all_uncert_df.groupby("Signal_ID")['Absolute Error'].max()),2)) + ' BPM')
    ax002.set_ylabel('Max Absolute Error')
    ax002.legend(loc='best')
    ax002.set_title("Max Absolute Error By Session")
    # Plot MAE for each Activity TYPE
    ax003 = fig.add_subplot(gs[0, 2])
    all_uncert_df.groupby("Activity")['Absolute Error'].mean().plot.bar(ax=ax003)
    ax003.axhline(np.mean(all_uncert_df.groupby("Activity")['Absolute Error'].mean()), ls='--',
                  label="Average @" + str(
                      round(np.mean(all_uncert_df.groupby("Activity")['Absolute Error'].mean()),2)) + ' BPM')
    ax003.set_ylabel('Mean Absolute Error')
    ax003.legend(loc='best')
    ax003.set_title("Mean Absolute Error By Activity")
    # Plot MAE for each Activity TYPE
    ax004 = fig.add_subplot(gs[0, 3])
    all_uncert_df.groupby("Fold", sort=False)['Absolute Error'].mean().plot.bar(ax=ax004)
    ax004.axhline(np.mean(all_uncert_df.groupby("Fold")['Absolute Error'].mean()), ls='--',
                  label="Average @" + str(
                      round(np.mean(all_uncert_df.groupby("Fold")['Absolute Error'].mean()),2)) + ' BPM')
    ax004.set_ylabel('Mean Absolute Error')
    ax004.legend(loc='best')
    ax004.set_title("Mean Absolute Error By Fold")
    
    # Plot heatmap of overlap between folds for train sets
    ax005 = fig.add_subplot(gs[1, 0])
    data, matrix = get_percentage_overlap_of_fold_sets(unique_check_train)
    sns.heatmap(data, mask=matrix, cmap="RdYlBu", ax=ax005)
    ax005.set_title("% Overlap Between Training Sets for Each Fold")
    # Plot heatmap of overlap between folds for val sets
    ax006 = fig.add_subplot(gs[1, 1])
    data, matrix = get_percentage_overlap_of_fold_sets(unique_check_val)
    sns.heatmap(data, mask=matrix, cmap="RdYlBu", ax=ax006)
    ax006.set_title("% Overlap Between Validation Sets for Each Fold")
    # Plot heatmap of overlap between folds for test sets
    ax007 = fig.add_subplot(gs[1, 2])
    data, matrix = get_percentage_overlap_of_fold_sets(unique_check_test)
    sns.heatmap(data, mask=matrix, cmap="RdYlBu", ax=ax007)
    ax007.set_title("% Overlap Between Test Sets for Each Fold")
    # plot calibration plot
    ax007 = fig.add_subplot(gs[1, 3])
    avg_len_train_val_test = [average_len(unique_check_train), 
                              average_len(unique_check_val), 
                             average_len(unique_check_test)]
    avg_len_names = ["Train", "Validation", "Test"]
    ax007.bar(avg_len_names,avg_len_train_val_test)
    ax007.set_ylabel('Number of Samples')
    ax007.set_title('Average Size of Each Data Split')
    
    ax008 = fig.add_subplot(gs[2, 0])
    sns.scatterplot(x='Truth', y='Absolute Error', data=all_uncert_df, ax=ax008)
    ax008.set_title('Relationship Between Truth and Absolute Error')
    # Plot relationship between SNR and AE
    ax009 = fig.add_subplot(gs[2, 1])
    sns.scatterplot(x='SNR', y='Absolute Error', data=all_uncert_df, ax=ax009)
    ax009.set_title('Relationship Between Est. SNR and Absolute Error')
    # Plot relationship between SNR and AE
    ax010 = fig.add_subplot(gs[2, 2])
    sns.boxplot(x='Activity', y='Absolute Error', data=all_uncert_df, showfliers=False, orient='v', ax=ax010)
    ax010.set_title('Relationship Between Activity and Absolute Error')
    # Plot bia/var
    ax011 = fig.add_subplot(gs[2, 3])
    sns.barplot(x='Fold', y='Session_Bias', data=df_session_specific, ax=ax011)
    ax011.axhline(np.mean(df_session_specific['Session_Bias']), ls='--',
                  label="Average @" + str(
                      round(np.mean(df_session_specific['Session_Bias']),2)) + ' BPM')
    ax011.set_xlabel('Fold')
    ax011.set_ylabel('Bias')
    ax011.set_title('Bias per Fold')
    ax011.legend(loc='best')

    ax012 = fig.add_subplot(gs[3, 0])
    uct.plot_calibration(y_pred, y_std, y_true, ax=ax012)
    # plot adversarial group calibration plot
    ax013 = fig.add_subplot(gs[3, 1])
    uct.plot_adversarial_group_calibration(y_pred, y_std, y_true, ax=ax013)
    ax014 = fig.add_subplot(gs[3, 2])
    sns.barplot(x='Fold', y='Session_Miscal_Area', data=df_session_specific, ax=ax014)
    ax014.axhline(np.mean(df_session_specific['Session_Miscal_Area']), ls='--',
                  label="Average @" + str(
                      round(np.mean(df_session_specific['Session_Miscal_Area']),2)) + ' BPM')
    ax014.set_xlabel('Fold')
    ax014.set_ylabel('Miscalibration Area')
    ax014.set_title('Miscalibration Area per Fold')
    ax014.legend(loc='best')
    
    ax015 = fig.add_subplot(gs[3, 3])
    sns.barplot(x='Fold', y='Session_Var', data=df_session_specific, ax=ax015)
    ax015.axhline(np.mean(df_session_specific['Session_Var']), ls='--',
                  label="Average @" + str(
                      round(np.mean(df_session_specific['Session_Var']),2)) + ' BPM')
    ax015.set_xlabel('Fold')
    ax015.set_ylabel('Variance')
    ax015.set_title('Variance per Fold')
    ax015.legend(loc='best')
    # plot sharpness plot
    ax016 = fig.add_subplot(gs[4, 0])
    uct.plot_sharpness(y_std, ax=ax016)
    ax017 = fig.add_subplot(gs[4, 1])
    sns.barplot(x='Fold', y='Session_Sharpness', data=df_session_specific, ax=ax017)
    ax017.axhline(np.mean(df_session_specific['Session_Sharpness']), ls='--',
                  label="Average @" + str(
                      round(np.mean(df_session_specific['Session_Sharpness']),2)) + ' BPM')
    ax017.set_xlabel('Fold')
    ax017.set_ylabel('Sharpness')
    ax017.set_title('Sharpness per Fold')
    # plot boxplot of activity type and epistemic uncert
    ax018 = fig.add_subplot(gs[4, 2])
    sns.boxplot(x='Activity', y='Epistemic', data=all_uncert_df, showfliers=False, orient='v', ax=ax018)
    ax0185 = ax018.twinx()
    sns.countplot(x='Activity', data=all_uncert_df, alpha=0.5, ax=ax0185)
    ax0185.tick_params('x', labelrotation=30)
    ax018.set_title('Relationship Between Activity and Epistemic Uncertainty')
    # plot boxplot of activity type and Aleotoric uncert
    ax019 = fig.add_subplot(gs[4, 3])
    sns.boxplot(x='Activity', y='Aleotoric', data=all_uncert_df, showfliers=False, orient='v', ax=ax019)
    ax019.tick_params('x', labelrotation=30)
    ax019.set_title('Relationship Between Activity and Aleotoric Uncertainty')
    # plot relationship between Aleotoric & Epistemic uncert
    ax020 = fig.add_subplot(gs[5, 0])
    sns.scatterplot(x='SNR', y='Aleotoric', data=all_uncert_df, ax=ax020)
    ax020.set_title('Relationship Between Est. SNR and Aleotoric Uncertainty')
    ax021 = fig.add_subplot(gs[5, 1])
    sns.scatterplot(x='SNR', y='Epistemic', data=all_uncert_df, ax=ax021)
    ax021.set_title('Relationship Between Est. SNR and Epistemic Uncertainty')
    ax021.set_yscale('log')
    ax022 = fig.add_subplot(gs[5, 2])
    ax022.set_yscale('log')
    sns.scatterplot(x='Aleotoric', y='Epistemic', data=all_uncert_df, ax=ax022)
    ax022.set_title('Relationship Between Aleotoric and Epistemic Uncertainty')
    
    gs.tight_layout(fig)

    # either save file as svg or png
    if publication:
        fig_format = 'svg'
    else:
        fig_format = 'png'
    fig_name = path + '/overall.' + fig_format
    fig.savefig(fig_name, format=fig_format, dpi=fig.dpi, bbox_inches='tight',
                bbox_extra_artists=[my_suptitle])
    # display graph during execution or not
    if display:
        plt.show()
        time.sleep(10)
        plt.close(fig)
        output.clear()

def get_overall_stats(path):
    """
    Gets overall stats of the performance of the model

    Args:
        path: The path of the experiment.

    Returns:
        all_uncert_df: Dataframe containing the uncertainty metrics.
        all_calibration_df: Dataframe containing the calibration metrics.
        df_session_specific: Dataframe containing session/fold specific metrics.
        df_overall: Dataframe containing overall metrics to compare with other models.
    """
    # init lists to store metrics
    calibration_results = []
    fold = []
    sess_mae = []
    sess_max_ae = []
    sess_ma = []
    sess_sharp = []
    sess_expected_loss = []
    sess_bias = []
    sess_var = []
    cali_filename = '*results_cali.csv'
    uncert_filename = '*results_uncert.csv'
    sess_spec_file_name = "results_fold_specific.csv"
    overall_file_name = "results_overall.csv"
    #get all the fold results into 1 df
    for filename in os.listdir(path):
        if fnmatch.fnmatch(filename, cali_filename):
            calibration_results.append(filename)
    dfs = list()
    for i, f in enumerate(calibration_results):
        data = pd.read_csv(path + "/" + f)
        #store fold name in df
        data['Fold'] = re.findall(r'\d+', f)[0]
        #get metrics for computing calibration metrics
        y_true = data['Truth'].to_numpy()
        y_pred = data['Mean_Value'].to_numpy()
        y_std = data['STD_Value'].to_numpy()
        #compute calibration metrics
        sess_sharp.append(uct.metrics_calibration.sharpness(y_std))
        sess_ma.append(uct.metrics_calibration.miscalibration_area(y_pred, y_std, y_true))
        dfs.append(data)
    #concat all dfs to one df
    all_calibration_df = pd.concat(dfs, ignore_index=True)
    # get all calib result files into one df
    uncert_results = []
    for filename in os.listdir(path):
        if fnmatch.fnmatch(filename, uncert_filename):
            uncert_results.append(filename)
    dfs1 = list()
    for i, f in enumerate(uncert_results):
        data = pd.read_csv(path + "/" + f)
        # store fold name in df
        data['Fold'] = re.findall(r'\d+', f)[0]
        fold.append(re.findall(r'\d+', f)[0])
        #compute fold specific metrics
        sess_mae.append(np.mean(data["Absolute Error"]))
        sess_max_ae.append(np.max(data["Absolute Error"]))
        all_pred = data["Predictive_Mean"].to_numpy()
        y_test = data["Truth"].to_numpy()
        #compute bias + variance of folds
        main_predictions = np.mean(all_pred, axis=0)
        sess_bias.append(np.sum((main_predictions - y_test) ** 2) / y_test.size)
        sess_var.append(np.sum((main_predictions - all_pred) ** 2) / all_pred.size)
        dfs1.append(data)
    # get all uncertainty result files into one df
    all_uncert_df = pd.concat(dfs1, ignore_index=True)
    #store fold specific metrics in dict to make into df
    fold_specific_metrics = []
    for i, session in enumerate(fold):
      session_metrics = {
          "Fold": fold[i],
          "Session_MAE": sess_mae[i],
          "Session_Max_AE": sess_max_ae[i],
          "Session_Miscal_Area": sess_ma[i],
          "Session_Sharpness": sess_sharp[i],
          "Session_Bias": sess_bias[i],
          "Session_Var": sess_var[i]
      }
      fold_specific_metrics.append(session_metrics)
    # store overall metrics in dict to make into df
    overall_dict = {
        "Exp_Name": path.split("/Exp/",1)[1],
        "MAE": np.mean(sess_mae),
        "MAE_std": np.std(sess_mae),
        "Max_Abs_Error": np.mean(sess_max_ae),
        "Max_Abs_Error_std": np.std(sess_max_ae),
        "Miscal_Area": np.mean(sess_ma),
        "Miscal_Area_std": np.std(sess_ma),
        "Sharpness": np.mean(sess_sharp),
        "Sharpness_std": np.std(sess_sharp),
        "Bias": np.mean(sess_bias),
        "Bias_std": np.std(sess_bias),
        "Var": np.mean(sess_var),
        "Var_std": np.std(sess_var)
    }
    #convert list of dicts to df
    df_session_specific = pd.DataFrame(fold_specific_metrics)
    df_overall = pd.DataFrame([overall_dict])
    #save both dfs are csv files
    df_session_specific.to_csv(path + "/" + str(sess_spec_file_name), index=False)
    df_overall.to_csv(path + "/" + str(overall_file_name), index=False)
    return all_uncert_df, all_calibration_df, df_session_specific, df_overall

def save_pickle(PATH, EXP_NAME, Name, pkl_list):
    """
    Save list as pickle.

    Args:
        PATH: The path of the experiment.
        EXP_NAME: The name of the experiment.
        Name: The name of the file.
        pkl_list: The list to pickle.
    """
    outfile = open(PATH + "/" + EXP_NAME + "/" + Name, 'wb')
    pickle.dump(pkl_list, outfile)
    outfile.close()

def repeating_values(x):
    """
    Get the results from the unseen session data.

    Args:
        x: list of activities.

    Returns:
        start_indexes: list of indices where the activity type starts.
        activity: the activity name related to that index.
    """
    start_indexes = []
    activity = []
    cur_idx = 0
    i = 1
    while i < len(x):
        if x[cur_idx] == x[i]:
            start_indexes.append(cur_idx)
            activity.append(x[cur_idx])
            # Increase value of i up to the index where there is different value
            while i < len(x):
                if x[cur_idx] != x[i]:
                    # Set cur_idx to the index of different value
                    cur_idx = i
                    i += 1
                    break
                i += 1
        else:
            cur_idx = i
            i += 1
    return start_indexes, activity

def Negative_Log_Likelihood(y, distribution):
    """
    Get NLL of an observation given a distribution.

    Args:
        y: an observation.
        distribution: a fitted distribution.

    Returns:
        -distribution.log_prob(y): the negative log likelihood of y given distribution.
    """
    return -distribution.log_prob(y)

def normal_sp(params):
    """
    Converts model outputs to Normal Distribution

    Args:
        params: list of parameters outputted from model.

    Returns:
        tfd.Normal: A normal distribution.
    """
    return tfd.Normal(loc=params[:, 0:1], scale=1e-3 + tf.math.softplus(0.05 * params[:, 1:2]))

def normal_sp_predict(params):
    """
    Converts model outputs to Normal Distribution

    Args:
        params: list of parameters outputted from model.

    Returns:
        tfd.Normal: A normal distribution.
    """
    return tfd.Normal(loc=params[0], scale=1e-3 + tf.math.softplus(0.05 * params[1]))

def train_model_LOSO(data, PATH, EXP_NAME, num_epochs, batch_sze, stratified, 
                     test_sze, num_predict_samples, band_pass_low, band_pass_high, 
                     band_pass_order, resample_fs, display, publication, 
                     SENSOR_FILTER_SIZE, SENSOR_NUM_FILTER, SENSOR_DROPOUT, 
                     GLOBAL_FILTER_SIZE, GLOBAL_NUM_FILTER, 
                     GLOBAL_DROPOUT, RECURRENT_UNITS, 
                     RECURRENT_DROPOUT, MAXPOOLING):
    """
    Preprocesses data, trains the model, displays training performance, predicts values of unseen session,
    displays the performance of uncertainty metrics. Repeats for each fold in dataset then displays overall
    performance.

    Args:
        data: The raw data file.
        PATH: The path of the experiment folder.
        EXP_NAME: The experiment name.
        num_epochs: The number of training epochs.
        batch_sze: The batch size of the model.
        stratified: Whether the train/val sets are stratified on activity type.
        test_sze: The percentage of the data used for validation set.
        num_predict_samples: The number of samples from the predictive distribution.
        band_pass_low: The lower cutoff frequency of the bandpass filter.
        band_pass_high: The higher cutoff frequency of the bandpass filter.
        band_pass_order: The order of the bandpass filter.
        resample_fs: The desired sampling frequency.
        display: Whether to display inline during training.
        publication: Whether to export as svg or png format.
        SENSOR_FILTER_SIZE: The filter size of the Sensor Specific Convolutional block.
        SENSOR_NUM_FILTER: The number of filters in the Sensor Specific Convolutional block.
        SENSOR_DROPOUT: The dropout rate for the Sensor Specific Convolutional block.
        GLOBAL_FILTER_SIZE: The filter size of the Global Convolutional block.
        GLOBAL_NUM_FILTER: The number of filters in the Global Convolutional block.
        GLOBAL_DROPOUT: The dropout rate for the Global Convolutional block.
        RECURRENT_UNITS: The number of units in the Recurrent block.
        RECURRENT_DROPOUT: The dropout rate for the Recurrent block.
        MAXPOOLING: The max pooling for the Convolutional blocks.
    """
    # preprocess dataset into df
    print("Preprocessing Dataset: ")
    DATASET = preprocess_dataset(data, band_pass_low, band_pass_high, 
                                 band_pass_order, resample_fs)
    unique_check_train = []
    unique_check_val = []
    unique_check_test = []
    train_times = []
    predict_times = []
    path = PATH + str(EXP_NAME)
    SUBJECT = DATASET["Signal_ID"].unique().tolist()
    random.shuffle(SUBJECT, random.random)
    model_print = 0
    # for each fold in dataset
    for fold, test_subject in enumerate(SUBJECT):
        model_print += 1
        # get train/val/test split
        df_train, df_val, df_test = get_dataset_splits_loso(DATASET, stratified, 
                                                            test_sze, test_subject)

        df_train["Dataset"] = "Train"
        df_val["Dataset"] = "Validation"
        df_test["Dataset"] = "Test"
        # get data in correct format for model
        PPG_Train = np.stack(df_train['PPG'], axis=0)
        ACC_Train = np.stack(df_train['ACC'], axis=0)
        Truth_Train = np.stack(df_train['Truth'], axis=0)

        PPG_Test = np.stack(df_test['PPG'], axis=0)
        ACC_Test = np.stack(df_test['ACC'], axis=0)
        Truth_Test = np.stack(df_test['Truth'], axis=0)

        PPG_Val = np.stack(df_val['PPG'], axis=0)
        ACC_Val = np.stack(df_val['ACC'], axis=0)
        Truth_Val = np.stack(df_val['Truth'], axis=0)

        # callbacks
        es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=70,
                                              verbose=1, restore_best_weights=False)
        tqdm_callback = tfa.callbacks.TQDMProgressBar()
        modelname = path + '/fold' + str(fold) + '.h5'
        rlronp = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                      factor=0.9, patience=5,
                                                      verbose=1)
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            modelname,
            monitor='val_loss',
            save_best_only=True,
            verbose=1, mode='min')
       
            # init model & optimiser
        model = make_DeepPulse(SENSOR_FILTER_SIZE, SENSOR_NUM_FILTER, 
                               SENSOR_DROPOUT, GLOBAL_FILTER_SIZE, 
                               GLOBAL_NUM_FILTER, GLOBAL_DROPOUT, 
                               RECURRENT_UNITS, RECURRENT_DROPOUT, MAXPOOLING)
        callbacks = [es, rlronp, checkpoint_callback, tqdm_callback]
        OPT = tf.optimizers.Nadam()
        model.compile(loss=Negative_Log_Likelihood, optimizer=OPT, metrics=['mae'])
        # train model
        start_training_time = time.time()
        print("Fit model on training data fold ", fold, "/", len(SUBJECT))
        history = model.fit(
            [PPG_Train, ACC_Train],
            Truth_Train,
            batch_size=batch_sze,
            epochs=num_epochs,
            validation_data=([PPG_Val, ACC_Val], Truth_Val),
            verbose=0,
            callbacks=callbacks)
        time.sleep(2)
        output.clear()
        # load best performing weights
        model.load_weights(modelname)
        # use trained model to predict on unseen session
        eval_history = model.evaluate(
            [PPG_Test, ACC_Test],
            Truth_Test,
            batch_size=batch_sze, verbose=0,
            callbacks=[tqdm_callback])
        end_training_time = time.time()
        train_time = end_training_time - start_training_time
        output.clear()
        # store windows used in each set of data to perform similarity analysis
        train_set_uiq = list(df_train.Signal_ID.astype(str).str.cat(df_train.Window_ID.astype(str), sep='w'))
        val_set_uiq = list(df_val.Signal_ID.astype(str).str.cat(df_val.Window_ID.astype(str), sep='w'))
        test_set_uiq = list(df_test.Signal_ID.astype(str).str.cat(df_test.Window_ID.astype(str), sep='w'))
        unique_check_train.append(train_set_uiq)
        unique_check_val.append(val_set_uiq)
        unique_check_test.append(test_set_uiq)
        # save model arch image
        if model_print == 1:
            tf.keras.utils.plot_model(
                make_DeepPulse(SENSOR_FILTER_SIZE, SENSOR_NUM_FILTER, 
                               SENSOR_DROPOUT, GLOBAL_FILTER_SIZE, 
                               GLOBAL_NUM_FILTER, GLOBAL_DROPOUT, 
                               RECURRENT_UNITS, RECURRENT_DROPOUT, MAXPOOLING),
                to_file=path + "/model.png",
                show_shapes=True,
            )
            # save experiment specific parameters
            with open(path + "/variables.txt", "w") as f:
                f.write("PREPROCESSING VARIABLES:" + "\n \n" + "BAND_PASS_LOW = "
                 + str(band_pass_low) + "\n" + "BAND_PASS_HIGH = " + 
                 str(band_pass_high) + "\n" + "BAND_PASS_ORDER = " + 
                 str(band_pass_order) + "\n" + "RESAMPLE_FS = " + str(resample_fs)
                  + "TRAINING VARIABLES:" + "\n \n"  + "CV = LOSO \n"  + "NUM_EPOCHS = " + 
                  str(num_epochs) + "\n" 
                + "BATCH_SIZE = " + str(batch_sze) + "\n" 
                + "STRATIFIED = " + str(stratified) + "\n" + "VAL_SET_SIZE = " 
                + str(test_sze) + "\n" + "NUM_PREDICT_SAMPLES = " + 
                str(num_predict_samples)+ "\n" + "ARCHITECTURAL VARIABLES:" + 
                "\n \n" + "SENSOR_FILTER_SIZE = " + str(SENSOR_FILTER_SIZE) + "\n" + 
                "SENSOR_NUM_FILTER = " + str(SENSOR_NUM_FILTER) +
                 "\n"+ "SENSOR_DROPOUT = " 
                + str(SENSOR_DROPOUT) + "\n"+ "SENSOR_CONV_TYPE = " + str(SENSOR_CONV_TYPE) 
                + "\n" + "GLOBAL_FILTER_SIZE = " + str(GLOBAL_FILTER_SIZE)
                  + "\n"+ "GLOBAL_NUM_FILTER = " + str(GLOBAL_NUM_FILTER) + "\n"+ 
                "GLOBAL_DROPOUT = " + str(GLOBAL_DROPOUT) 
                + "\n"+ "GLOBAL_CONV_TYPE = " + str(GLOBAL_CONV_TYPE) + 
                "\n"+ "RECURRENT_UNITS = " + str(RECURRENT_UNITS)
                  + "\n" + "RECURRENT_DROPOUT = " + str(RECURRENT_DROPOUT) 
                  + "\n" + "MAXPOOLING = " + str(MAXPOOLING))
                f.close()
        start_predict_time = time.time()
        # get training graph
        df_result = pd.concat([df_train, df_val, df_test]).reset_index(drop=True)
      
        display_training_performance(df_result, history, eval_history, fold, path, display, publication)
        # get calibration graph
        calibration_df, uncert_df = get_test_results(model, df_test, fold, modelname, num_predict_samples)
        display_calibration_performance(calibration_df, uncert_df, fold, path, display, publication)
        end_predict_time = time.time()
        predict_time = end_predict_time - start_predict_time
        train_times.append(train_time)
        predict_times.append(predict_time)
        with open(path + "/times.txt", "a") as f:
          f.write("FOLD: " + str(fold)+ "\n"+ "Train Time: " + str(train_time)+ "\n"
          + "Predict Time: " + str(predict_time)+ "\n")
          f.close()
        # delete fold data
        del history
        del eval_history
        del model
    save_pickle(PATH, EXP_NAME, "unique_check_train", unique_check_train)
    save_pickle(PATH, EXP_NAME, "unique_check_val", unique_check_val)
    save_pickle(PATH, EXP_NAME, "unique_check_test", unique_check_test)
    # get overall graph
    display_overall_performance(path, unique_check_train, unique_check_val, unique_check_test, display, publication)
    display_overall_performance(path, unique_check_train, unique_check_val, unique_check_test, display, publication)

def average_len(l):
    """
    Computes the average length of a list of lists
    Taken from: https://stackoverflow.com/a/15772649

    Args:
        l: list of lists.

    Returns:
        sum(map(len, l))/float(len(l)): Average length of a list of lists.
    """

    return sum(map(len, l))/float(len(l))
    

# Model Training

In [ ]:
make_DeepPulse(SENSOR_FILTER_SIZE, SENSOR_NUM_FILTER, SENSOR_DROPOUT, 
                   GLOBAL_FILTER_SIZE, GLOBAL_NUM_FILTER, 
                   GLOBAL_DROPOUT, RECURRENT_UNITS, 
                   RECURRENT_DROPOUT, MAXPOOLING).summary()

In [ ]:
for idx, dataset in enumerate(DATA):
  train_model_LOSO(dataset, PATH, EXP_NAME[idx], NUM_EPOCHS, BATCH_SIZE, STRATIFIED, 
              TEST_SIZE, NUM_PREDICT_SAMPLES, BAND_PASS_LOW, BAND_PASS_HIGH, 
              BAND_PASS_ORDER, RESAMPLE_FS, DISPLAY, PUBLICATION, 
              SENSOR_FILTER_SIZE, SENSOR_NUM_FILTER, SENSOR_DROPOUT, 
              GLOBAL_FILTER_SIZE, GLOBAL_NUM_FILTER, GLOBAL_DROPOUT, 
              RECURRENT_UNITS, RECURRENT_DROPOUT, MAXPOOLING)